In [1]:
# Cell 1: Imports and Setup
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler  # If not already scaled
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import joblib  # For saving scaler if needed

# Set seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Load preprocessed data (adjust path if final_output.csv is elsewhere)
df = pd.read_csv('../../outputs/final_output.csv')  # From preprocessing outputs/
X = df.drop('selling_price', axis=1).values  # Features as array for Keras
y = df['selling_price'].values  # Target

# Scale features (if not done in preprocessing; MLP benefits from it)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
     |█████████████▎                  | 82.8 MB 209 kB/s eta 0:09:23